## Generator imion, nazwisk, e-mailu, numerów telefonów pracowników

### 1)Odczytywanie popularnych imion i nazwisk żeńskich i męskich.

In [1]:
import pandas as pd
import pathlib as path


# Znajdź katalog z imionami i nazwiskami.
name_dir_path: path.Path = path.Path().cwd()/"Imiona i nazwiska"

def read_names(first_names_file:str, last_names_file:str, dir_path:path.Path) -> tuple[pd.Series, pd.Series]:
    """Funkcja odczytuje pliki zawierające najpopularniejsze polskie imiona i nazwiska. Wczytuje je do szeregu danych typu pandas"""
    # Odczytaj imiona oraz nazwiska.
    first_names: pd.DataFrame = pd.read_excel(dir_path/first_names_file  ,"Arkusz1",
                                                usecols = [0,2]
                                                )
    last_names: pd.DataFrame = pd.read_excel(dir_path/last_names_file, "Arkusz1",
                                                nrows = 5_000) # Wierszy w arkuszu jest bardzo dużo (ponad 380 000). 
                                                                # Odczytaj tylko pewną część wierszy.
                                                                

    # Bierzemy pod uwagę jedynie najpopularniejsze imiona i nazwiska.
    first_names = first_names.loc[first_names["LICZBA_WYSTĄPIEŃ"] > 500, "IMIĘ_PIERWSZE"]
    last_names = last_names.loc[last_names["Liczba"] > 500, "Nazwisko aktualne"] 

    # Skróc nazwy kolumn.
    first_names.columns = ['imię']
    last_names.columns = ['nazwisko']

    return first_names, last_names

# Wczytaj męskie imiona i nazwiska.
male_first_names, male_last_names = read_names("imiona_meskie.xlsx", "nazwiska_meskie.xlsx", name_dir_path)

# Wczytaj żęńskie imiona i nazwiska.
female_first_names, female_last_names = read_names("imiona_zenskie.xlsx", "nazwiska_zenskie.xlsx", name_dir_path)


###  Tworzenie danych osobowych.

In [2]:
import numpy as np

n_female, n_male = 25, 21 # Ustal ile będzie pracowniczek i pracowników.


def generate_personal_data(first_names:pd.Series, last_names:pd.Series, n:int, email_domain:str) -> pd.DataFrame:
    """Funkcja generuje dane osobiste pracowników. Tworzone są pary imię-nazwisko, numer telefonu oraz adres e-mail firmowy.

    """

    # Przygotuj wyjściową ramkę danych.
    workers_df: pd.DataFrame = pd.DataFrame(data = {"imię":pd.Series(dtype = pd.StringDtype()),
                                                    "nazwisko": pd.Series(dtype = pd.StringDtype()), 
                                                    "numer telefonu":pd.Series(dtype = pd.StringDtype()), 
                                                    "adres email": pd.Series(dtype = pd.StringDtype())},
                                                
    )

    # Lista wszystkich cyfr. Przyda się przy generowaniu numerów telefonów.
    digits:list[str] = ["0","1","2","3","4","5","6","7","8","9"]

    # Stwórz n fikcyjnych różnych osób.
    for id in range(n):
        rand_first_name_idx:int = np.random.randint(0, first_names.shape[0])
        rand_last_name_idx:int = np.random.randint(0, last_names.shape[0])

        rand_first_name:str = first_names[rand_first_name_idx].capitalize() # Wylosuj imię
        rand_last_name:str = last_names[rand_last_name_idx].capitalize()  # Wylosuj nazwisko.

        # Wygeneruj losowy numer telefonu nie zaczynający się od 0.
        phone_number:np.array = np.random.choice(digits, 9)

        # Jeżeli wylosowaliśmy zero, podmień cyfrę.
        if phone_number[0] == "0":
            phone_number[0] = np.random.choice(digits[1:], 1)[0]

        phone_number: str = "".join(phone_number)

        # Stwórz email dla tego pracownika
        email:str = rand_first_name.lower() + rand_last_name.lower() + "".join(np.random.choice(digits, 3)) +"@" + email_domain
        

        workers_df.loc[id, :] = [rand_first_name, rand_last_name, phone_number, email]

    return workers_df


# Wygeneruj dane osobiste dla pracowniczek
email_domain = "nowowiejska.com"
female_workers_data:pd.DataFrame = generate_personal_data(female_first_names, female_last_names, n_female, email_domain)
male_workers_data: pd.DataFrame = generate_personal_data(male_first_names, male_last_names, n_male, email_domain)

# Połącz kobiecych i męskich pracowników.
workers_data:pd.DataFrame = pd.concat(objs = [female_workers_data, male_workers_data], axis = 0).reset_index(drop = True)

workers_data.insert(0, "ID", range(1, n_female+n_male+1))

workers_data

,ID,imię,nazwisko,numer telefonu,adres email
0,1,Alicja,Lach,347664714,alicjalach732@nowowiejska.com
1,2,Helena,Błaszczyk,858628067,helenabłaszczyk833@nowowiejska.com
2,3,Jaśmina,Szatan,546127304,jaśminaszatan289@nowowiejska.com
3,4,Zofia,Marks,778816329,zofiamarks385@nowowiejska.com
4,5,Kornelia,Górnicka,133023089,korneliagórnicka224@nowowiejska.com
5,6,Wanda,Janiszewska,325393012,wandajaniszewska518@nowowiejska.com
6,7,Lena,Bogus,778217815,lenabogus965@nowowiejska.com
7,8,Jagoda,Cegłowska,218558013,jagodacegłowska666@nowowiejska.com
8,9,Milena,Kostka,751936053,milenakostka294@nowowiejska.com
9,10,Sara,Kołacz,906449949,sarakołacz866@nowowiejska.com
